<a href="https://colab.research.google.com/github/vangogh1803/face_rec/blob/main/face_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = "/content/drive/MyDrive/face_proj"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, Dataset
import os
import numpy as np
from datasets import load_dataset
from PIL import Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)


Using: cpu


In [ ]:
!pip install -q datasets


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list | head


ref                                                  title                                           size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------  ----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
neurocipher/heartdisease                             Heart Disease                                   3491  2025-12-11 15:29:14.327000           2114        204  1.0              
ahmeduzaki/wind-and-solar-energy-production-dataset  Wind & Solar Energy Production Dataset        395372  2026-01-02 21:06:22.780000              0         35  1.0              
kundanbedmutha/exam-score-prediction-dataset         Exam Score Prediction Dataset                 325454  2025-11-28 07:29:01.047000           5863        249  1.0              
guriya79/heart-failure-prediction-dataset            Heart Failure Clinical Records Study            4067

In [ ]:
!kaggle datasets download -d kenny3s/casia-webface

Dataset URL: https://www.kaggle.com/datasets/kenny3s/casia-webface
License(s): MIT
casia-webface.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!kaggle datasets download -d jessicali9530/lfw-dataset


Dataset URL: https://www.kaggle.com/datasets/jessicali9530/lfw-dataset
License(s): other
  0% 0.00/112M [00:00<?, ?B/s]
100% 112M/112M [00:00<00:00, 1.31GB/s]


In [ ]:
!unzip /content/data/lfw-dataset.zip -d /content/data


unzip:  cannot find or open /content/data/lfw-dataset.zip, /content/data/lfw-dataset.zip.zip or /content/data/lfw-dataset.zip.ZIP.


In [ ]:
!unzip /content/casia-webface.zip -d /content/data


Archive:  /content/casia-webface.zip
replace /content/data/datasets/0000045/001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
one


In [ ]:
from collections import defaultdict
import random
from torch.utils.data import Subset

# Build identity → image indices map
id_to_indices = defaultdict(list)
for idx, lbl in enumerate(dataset.targets):
    id_to_indices[lbl].append(idx)

# Keep identities with >= 2 images
valid_ids = [lbl for lbl, idxs in id_to_indices.items() if len(idxs) >= 2]
print("Valid identities:", len(valid_ids))

# Split identities
random.shuffle(valid_ids)
split = int(0.8 * len(valid_ids))

train_ids = valid_ids[:split]
val_ids   = valid_ids[split:]

# Build index lists
train_idx, val_idx = [], []

for lbl in train_ids:
    train_idx.extend(id_to_indices[lbl])

for lbl in val_ids:
    val_idx.extend(id_to_indices[lbl])

print("Train images:", len(train_idx))
print("Val images:", len(val_idx))

# Build datasets
train_dataset = Subset(dataset, train_idx)
val_dataset   = Subset(dataset, val_idx)


In [ ]:
train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader=DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
!pip install facenet-pytorch


In [ ]:
from facenet_pytorch import MTCNN
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mtcnn = MTCNN(
    image_size=224,
    margin=20,
    keep_all=False,
    device=device
)


In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class FaceDetectDataset(Dataset):
    def __init__(self, base_dataset, mtcnn):
        self.base_dataset = base_dataset
        self.mtcnn = mtcnn

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        img, label = self.base_dataset[idx]

        # ensure PIL image
        if isinstance(img, torch.Tensor):
            img = transforms.ToPILImage()(img)

        # detect face
        face = self.mtcnn(img)

        if face is None:
            # fallback: resize original image
            face = transforms.Resize((224,224))(img)
            face = transforms.ToTensor()(face)
        else:
            # face is already tensor
            face = face

        return face, label


# *Classification*

In [ ]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.Grayscale(num_output_channels=3),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
# ])
##augmentation
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([
        transforms.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2
        )
    ], p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
!find /content/data -maxdepth 3 -type d | head -n 20



/content/data
/content/data/casia_dev
/content/data/casia_dev/0000696
/content/data/casia_dev/0000981
/content/data/casia_dev/0000422
/content/data/casia_dev/0000948
/content/data/casia_dev/0000675
/content/data/casia_dev/0000260
/content/data/casia_dev/0000996
/content/data/casia_dev/0000304
/content/data/casia_dev/0000880
/content/data/casia_dev/0000570
/content/data/casia_dev/0000275
/content/data/casia_dev/0000670
/content/data/casia_dev/0000616
/content/data/casia_dev/0000615
/content/data/casia_dev/0000133
/content/data/casia_dev/0000394
/content/data/casia_dev/0000836
/content/data/casia_dev/0000373


In [ ]:
dataset_path = "/content/data/datasets"

dataset = datasets.ImageFolder(
    root=dataset_path,
    transform=transform
)

print("Total images:", len(dataset))
print("Total identities:", len(dataset.classes))


Total images: 452960
Total identities: 10575


In [ ]:
DEV_IDENTITIES = 300
IMAGES_PER_ID = 25
import os, shutil
from collections import defaultdict

SRC = "/content/data/datasets"
DST = "/content/data/casia_dev"

os.makedirs(DST, exist_ok=True)


In [ ]:
identities = sorted(os.listdir(SRC))[:DEV_IDENTITIES]

for ident in identities:
    src_id = os.path.join(SRC, ident)
    dst_id = os.path.join(DST, ident)
    os.makedirs(dst_id, exist_ok=True)

    imgs = os.listdir(src_id)[:IMAGES_PER_ID]
    for img in imgs:
        shutil.copy(
            os.path.join(src_id, img),
            os.path.join(dst_id, img)
        )

print("CASIA dev subset ready")


CASIA dev subset ready


In [ ]:
dataset_path = "/content/data/casia_dev"

dataset = datasets.ImageFolder(
    root=dataset_path,
    transform=transform
)

print("Images:", len(dataset))
print("Identities:", len(dataset.classes))


Images: 7156
Identities: 300


In [ ]:
train_size=int(0.8*len(dataset))
val_size=len(dataset)-train_size
train_dataset, val_dataset=torch.utils.data.random_split(
    dataset, [train_size, val_size]
  )


In [ ]:
from torch.utils.data import Subset
import random

labels = dataset.targets
id_to_idx = {}

for i, lbl in enumerate(labels):
    id_to_idx.setdefault(lbl, []).append(i)

ids = list(id_to_idx.keys())
random.shuffle(ids)

split = int(0.8 * len(ids))
train_ids = ids[:split]
val_ids = ids[split:]

train_idx, val_idx = [], []

for i in train_ids:
    train_idx.extend(id_to_idx[i])

for i in val_ids:
    val_idx.extend(id_to_idx[i])

train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
model=models.resnet18(pretrained=True)
model.fc=nn.Linear(model.fc.in_features, len(dataset.classes))
model=model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
#train now:
Epochs=3
for epoch in range(Epochs):
  model.train()
  running_loss=0
  correct=0
  total=0

  for images, labels in train_loader:
    images=images.to(device)
    labels=labels.to(device)

    optimizer.zero_grad()
    output=model(images)
    loss=criterion(output, labels)

    loss.backward()
    optimizer.step()

    running_loss+=loss.item()
    _, preds=torch.max(output, 1)
    correct+=(preds==labels).sum().item()
    total+=labels.size(0)

    train_acc=100*correct/total
    print(f"Epoch [{epoch+1}/{Epochs}] "
          f"Loss: {running_loss:.3f} | Train Acc: {train_acc:.2f}%")


Epoch [1/3] Loss: 5.936 | Train Acc: 0.00%
Epoch [1/3] Loss: 11.948 | Train Acc: 0.00%
Epoch [1/3] Loss: 18.245 | Train Acc: 0.00%
Epoch [1/3] Loss: 24.266 | Train Acc: 0.00%
Epoch [1/3] Loss: 30.409 | Train Acc: 0.00%
Epoch [1/3] Loss: 36.517 | Train Acc: 0.00%
Epoch [1/3] Loss: 42.386 | Train Acc: 0.00%
Epoch [1/3] Loss: 48.513 | Train Acc: 0.00%
Epoch [1/3] Loss: 54.614 | Train Acc: 0.00%
Epoch [1/3] Loss: 60.627 | Train Acc: 0.00%
Epoch [1/3] Loss: 66.614 | Train Acc: 0.00%
Epoch [1/3] Loss: 72.790 | Train Acc: 0.00%
Epoch [1/3] Loss: 78.918 | Train Acc: 0.00%
Epoch [1/3] Loss: 85.029 | Train Acc: 0.00%
Epoch [1/3] Loss: 90.874 | Train Acc: 0.00%
Epoch [1/3] Loss: 96.660 | Train Acc: 0.00%
Epoch [1/3] Loss: 102.620 | Train Acc: 0.00%
Epoch [1/3] Loss: 108.550 | Train Acc: 0.00%
Epoch [1/3] Loss: 114.440 | Train Acc: 0.00%
Epoch [1/3] Loss: 120.330 | Train Acc: 0.00%
Epoch [1/3] Loss: 126.379 | Train Acc: 0.15%
Epoch [1/3] Loss: 132.267 | Train Acc: 0.14%
Epoch [1/3] Loss: 138.235 |

# **image to embeddings**

# ResNet

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        _, preds = torch.max(output, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = 100 * correct / total
print("Validation Accuracy:", val_acc)


Validation Accuracy: 0.0


In [ ]:
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/face_proj/phase1_casia_magf.pth"
)
print("Model saved")


Model saved


In [ ]:
print(dataset.classes[:10])


['0000045', '0000099', '0000100', '0000102', '0000103', '0000105', '0000107', '0000108', '0000114', '0000117']


In [ ]:
import torch.nn as nn
from torchvision import models

base_model = models.resnet18(pretrained=True)

# Remove classifier entirely
base_model.fc = nn.Identity()

base_model = base_model.to(device)
base_model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
embeddings = []
labels = []

with torch.no_grad():
    for images, lbls in val_loader:
        images = images.to(device)
        feats = base_model(images)   # [B, 512]

        embeddings.append(feats.cpu())
        labels.append(lbls)

embeddings = torch.cat(embeddings).numpy()
labels = torch.cat(labels).numpy()

print("Embeddings shape:", embeddings.shape)


Embeddings shape: (2551, 512)


In [ ]:
from collections import defaultdict
import random
import numpy as np

# Map identity → embedding indices
id_to_indices = defaultdict(list)
for idx, lbl in enumerate(labels):
    id_to_indices[lbl].append(idx)

pairs = []
pair_labels = []

# Positive pairs (same identity)
for lbl, idxs in id_to_indices.items():
    if len(idxs) >= 2:
        i1, i2 = random.sample(idxs, 2)
        pairs.append((i1, i2))
        pair_labels.append(1)

# Negative pairs (different identities)
all_labels = list(id_to_indices.keys())

for _ in range(len(pairs)):
    l1, l2 = random.sample(all_labels, 2)
    i1 = random.choice(id_to_indices[l1])
    i2 = random.choice(id_to_indices[l2])
    pairs.append((i1, i2))
    pair_labels.append(0)

pairs = np.array(pairs)
pair_labels = np.array(pair_labels)

print("Total pairs:", len(pairs))


Total pairs: 140


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

scores = []

for i1, i2 in pairs:
    sim = cosine_similarity(
        embeddings[i1].reshape(1, -1),
        embeddings[i2].reshape(1, -1)
    )[0][0]
    scores.append(sim)

scores = np.array(scores)


In [ ]:
same_scores = scores[pair_labels == 1]
diff_scores = scores[pair_labels == 0]

print("Same-person mean similarity:", same_scores.mean())
print("Different-person mean similarity:", diff_scores.mean())


Same-person mean similarity: 0.8047906
Different-person mean similarity: 0.7520156


In [ ]:
##EER
from sklearn.metrics import roc_curve
import numpy as np

fpr, tpr, thresholds = roc_curve(pair_labels, scores, pos_label=1)
fnr = 1 - tpr

eer_idx = np.nanargmin(np.abs(fpr - fnr))
eer = fpr[eer_idx]

print("EER:", eer) #shows how many decisions are wrong



EER: 0.37142857142857144


# ResNet+ArcFace Loss function

In [ ]:
!pip install insightface


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, x, label):
        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros(cosine.size(), device=x.device)
        one_hot.scatter_(1, label.view(-1, 1), 1.0)

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


In [ ]:
from torchvision import models

num_classes = len(dataset.classes)

backbone = models.resnet18(pretrained=True)
backbone.fc = nn.Identity()
backbone = backbone.to(device)

arcface = ArcMarginProduct(
    in_features=512,
    out_features=num_classes,
    s=30.0,
    m=0.5
).to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    list(backbone.parameters()) + list(arcface.parameters()),
    lr=1e-4
)


In [ ]:
for epoch in range(15):   # 3–5 epochs is enough for now
    backbone.train()
    arcface.train()

    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        embeddings = backbone(images)
        logits = arcface(embeddings, labels)
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}] Loss: {total_loss:.3f}")


KeyboardInterrupt: 

In [ ]:
backbone.eval()

embeddings = []
labels = []

with torch.no_grad():
    for images, lbls in val_loader:
        images = images.to(device)
        feats = backbone(images)
        embeddings.append(feats.cpu())
        labels.append(lbls)

embeddings = torch.cat(embeddings).numpy()
labels = torch.cat(labels).numpy()

from sklearn.preprocessing import normalize
embeddings = normalize(embeddings)


In [ ]:
from collections import defaultdict
import random

id_to_indices = defaultdict(list)
for idx, lbl in enumerate(labels):
    id_to_indices[lbl].append(idx)

pairs = []
pair_labels = []   # 1 = same person, 0 = different

# Positive (same identity)
for lbl, idxs in id_to_indices.items():
    if len(idxs) >= 2:
        i1, i2 = random.sample(idxs, 2)
        pairs.append((i1, i2))
        pair_labels.append(1)

# Negative (different identity)
all_labels = list(id_to_indices.keys())

for _ in range(len(pairs)):
    l1, l2 = random.sample(all_labels, 2)
    i1 = random.choice(id_to_indices[l1])
    i2 = random.choice(id_to_indices[l2])
    pairs.append((i1, i2))
    pair_labels.append(0)

pairs = np.array(pairs)
pair_labels = np.array(pair_labels)

print("Total verification pairs:", len(pairs))


Total verification pairs: 120


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

scores = []

for i1, i2 in pairs:
    sim = cosine_similarity(
        embeddings[i1].reshape(1, -1),
        embeddings[i2].reshape(1, -1)
    )[0][0]
    scores.append(sim)

scores = np.array(scores)


IndexError: index 32 is out of bounds for dimension 0 with size 32

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(pair_labels, scores, pos_label=1)
fnr = 1 - tpr


In [ ]:
import numpy as np

eer_index = np.nanargmin(np.abs(fpr - fnr))
eer = fpr[eer_index]

print("EER:", eer)


EER: 0.45714285714285713


## ResNet+MagFace

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MagFace(nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        s=30.0,
        m_min=0.45,
        m_max=0.8,
        l_a=10,
        u_a=110
    ):
        super().__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.s = s

        self.m_min = m_min
        self.m_max = m_max
        self.l_a = l_a
        self.u_a = u_a

        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, embeddings, labels):
        # embedding norm = quality
        norm = torch.norm(embeddings, dim=1, keepdim=True).clamp(self.l_a, self.u_a)

        # adaptive margin
        margin = (
            self.m_min +
            (self.m_max - self.m_min) *
            (norm - self.l_a) / (self.u_a - self.l_a)
        )

        cosine = F.linear(F.normalize(embeddings), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - cosine ** 2 + 1e-6)
        phi = cosine * torch.cos(margin) - sine * torch.sin(margin)

        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1, 1), 1.0)

        output = one_hot * phi + (1.0 - one_hot) * cosine
        output *= self.s

        return output, norm


In [ ]:
from torchvision import models

num_classes = len(dataset.classes)

backbone = models.resnet18(pretrained=True)
backbone.fc = nn.Identity()
backbone = backbone.to(device)

magface = MagFace(
    in_features=512,
    out_features=num_classes
).to(device)
ce_loss = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(
    list(backbone.parameters()) + list(magface.parameters()),
    lr=1e-4,
    weight_decay=1e-4
)


In [ ]:
for epoch in range(10):
    backbone.train()
    magface.train()

    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        embeddings = backbone(images)
        logits, norms = magface(embeddings, labels)

        loss_id = ce_loss(logits, labels)

        # MagFace regularization (norm constraint)
        loss_reg = torch.mean((norms - magface.l_a) ** 2)

        loss = loss_id + 0.01 * loss_reg

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss:.3f}")


In [ ]:
val_dataset_fd = FaceDetectDataset(val_dataset, mtcnn)

from torch.utils.data import DataLoader
val_loader_fd = DataLoader(
    val_dataset_fd,
    batch_size=32,
    shuffle=False
)


In [ ]:
embeddings = []
labels = []

backbone.eval()
with torch.no_grad():
    for imgs, lbls in val_loader:
        imgs = imgs.to(device)
        feats = backbone(imgs)
        embeddings.append(feats.cpu())
        labels.append(lbls)

import torch
embeddings = torch.cat(embeddings).numpy()
labels = torch.cat(labels).numpy()

print("Embeddings shape:", embeddings.shape)
print("Labels shape:", labels.shape)


In [ ]:
from sklearn.preprocessing import normalize

# If embeddings is a torch tensor on GPU
embeddings = embeddings.detach().cpu().numpy()

# Now normalize
embeddings = normalize(embeddings)


In [ ]:
from collections import Counter

print("Val dataset size:", len(val_dataset))

val_labels = []
for _, lbl in val_dataset:
    val_labels.append(lbl)

label_counts = Counter(val_labels)

print("Number of identities in val:", len(label_counts))
print("Images per identity (val):", label_counts)
print("Identities with >=2 images:",
      sum(1 for v in label_counts.values() if v >= 2))


Val dataset size: 1459
Number of identities in val: 60
Images per identity (val): Counter({117: 25, 185: 25, 275: 25, 22: 25, 196: 25, 87: 25, 223: 25, 95: 25, 285: 25, 65: 25, 137: 25, 98: 25, 217: 25, 140: 25, 24: 25, 62: 25, 116: 25, 86: 25, 44: 25, 7: 25, 38: 25, 172: 25, 202: 25, 85: 25, 67: 25, 284: 25, 245: 25, 74: 25, 32: 25, 162: 25, 109: 25, 79: 25, 251: 25, 261: 25, 153: 25, 201: 25, 180: 25, 220: 25, 266: 25, 151: 25, 210: 25, 258: 25, 241: 25, 193: 25, 206: 25, 84: 25, 283: 25, 219: 25, 43: 25, 2: 25, 272: 25, 228: 25, 195: 25, 47: 25, 273: 23, 212: 21, 239: 18, 55: 16, 164: 16, 205: 15})
Identities with >=2 images: 60


Embeddings shape: (1441, 512)
Labels shape: (1441,)


In [ ]:
from collections import defaultdict
import numpy as np

id_to_indices = defaultdict(list)
for idx, lbl in enumerate(labels):
    id_to_indices[lbl].append(idx)

pairs = []
pair_labels = []

# --- Positive pairs (deterministic) ---
for lbl, idxs in id_to_indices.items():
    if len(idxs) >= 2:
        for i in range(len(idxs) - 1):
            pairs.append((idxs[i], idxs[i+1]))
            pair_labels.append(1)

# --- Negative pairs (deterministic, balanced) ---
all_labels = sorted(id_to_indices.keys())

neg_pairs = []
for i in range(len(all_labels)):
    for j in range(i+1, len(all_labels)):
        neg_pairs.append((
            id_to_indices[all_labels[i]][0],
            id_to_indices[all_labels[j]][0]
        ))

# Balance positives and negatives
neg_pairs = neg_pairs[:len(pairs)]

pairs.extend(neg_pairs)
pair_labels.extend([0] * len(neg_pairs))

pairs = np.array(pairs)
pair_labels = np.array(pair_labels)

print("Total pairs:", len(pairs))
print("Label distribution:", np.unique(pair_labels, return_counts=True))



Total pairs: 4214
Label distribution: (array([0, 1]), array([2107, 2107]))


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

scores = []

for i1, i2 in pairs:
    scores.append(
        cosine_similarity(
            embeddings[i1].reshape(1,-1),
            embeddings[i2].reshape(1,-1)
        )[0][0]
    )

scores = np.array(scores)
print("Scores length:", len(scores))


Scores length: 4214


In [ ]:
import numpy as np

print("pair_labels unique:", np.unique(pair_labels, return_counts=True))
print("scores stats:")
print("  min:", np.min(scores))
print("  max:", np.max(scores))
print("  mean:", np.mean(scores))
print("Any NaN in scores?", np.isnan(scores).any())


pair_labels unique: (array([0, 1]), array([2107, 2107]))
scores stats:
  min: 0.3294811
  max: 0.9994273
  mean: 0.65321547
Any NaN in scores? False


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_curve

#embeddings = normalize(embeddings)

# scores = []
# for i1, i2 in pairs:
#     scores.append(
#         cosine_similarity(
#             embeddings[i1].reshape(1,-1),
#             embeddings[i2].reshape(1,-1)
#         )[0][0]
#     )

# scores = np.array(scores)
# pair_labels = np.array(pair_labels)

fpr, tpr, _ = roc_curve(pair_labels, scores)
fnr = 1 - tpr
eer = fpr[np.nanargmin(np.abs(fpr - fnr))]

print("EER:", eer)


EER: 0.21547223540579022


In [ ]:
from sklearn.metrics import roc_curve
import numpy as np

fpr, tpr, thresholds = roc_curve(pair_labels, scores)
fnr = 1 - tpr

# EER
eer_idx = np.nanargmin(np.abs(fpr - fnr))
eer = fpr[eer_idx]

# FRR at FAR = 1%
target_far = 0.05
far_idx = np.argmin(np.abs(fpr - target_far))
frr_at_far = fnr[far_idx]

print("EER:", eer)
print("FRR @ FAR=1%:", frr_at_far)


EER: 0.21547223540579022
FRR @ FAR=1%: 0.6416706217370669


In [ ]:
!mkdir -p /content/drive/MyDrive/face_proj
